## Chapter 2

In [1]:
from platform import python_version

print(python_version())

3.9.13


In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier(
    [
        "I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",
    ]
)

/Users/celinelindeque/Documents/programming/ML/NLP/NLP/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-25 16:38:20.500077: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/Users/celinelindeque/Documents/programming/ML/NLP/NLP/lib/python3.9/site-packages/huggingface_hub/file

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

## Preprocessing

![Image of Yaktocat](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter2/full_nlp_pipeline.svg)

### Notes  
* Transformers can't process text directly and need it converted into a numerical format for processing
* To transform the text into numbers we use a tokenizer

* Tokenizers:  
    * words -> tokens
        * words, sub-words, or punctuation (!) into tokens
    * token -> mapped to an integer
        * each token (letters) is mapped to an integer (number)
    * also, may add additional inputs which might be useful for the model 

* This process must be done in an identical fashion as to how the pre-trained model was preprocessed
* HuggingFace's transformer library figures this out for you, when using a pre-trained model

* "...we use the *AutoTokenizer* class and its *from_pretrained()* method. Using the *checkpoint name* of our model, it will automatically fetch the data associated with the model’s tokenizer and cache it (so it’s only downloaded the first time you run the code below)."

"...The default checkpoint of the *sentiment-analysis* pipeline is *distilbert-base-uncased-finetuned-sst-2-english*"

In [3]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

* "Transformer models only accept *tensors* as input. If this is your first time hearing about tensors, you can think of them as *NumPy arrays* instead. A NumPy array can be a scalar (0D), a vector (1D), a matrix (2D), or have more dimensions. It’s *effectively* a tensor; other ML frameworks’ tensors behave similarly, and are usually as simple to instantiate as NumPy arrays."

In [4]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


* "The output itself is a dictionary containing two keys, *input_ids* and *attention_mask*.  
* input_ids contains two rows of integers (one for each sentence) that are the unique identifiers of the tokens in each sentence"

## The Model

In [5]:
from transformers import TFAutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


* "This architecture contains only the base Transformer module: given some inputs, it outputs what we’ll call hidden states, also known as features. For each model input, we’ll retrieve a high-dimensional vector representing *the contextual understanding of that input by the Transformer model.*"  

* "The vector output by the Transformer module is usually large. It generally has three dimensions:
    * Batch size: The number of sequences processed at a time (2 in our example).
    * Sequence length: The length of the numerical representation of the sequence (16 in our example).
    * Hidden size: The vector dimension of each model input."
        * "The hidden size can be very large (768 is common for smaller models, and in larger models this can reach 3072 or more)."

In [6]:
inputs

{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}

In [7]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 16, 768)


## Model Heads